In [1]:
from source.regency import Regency
from source.mapping import Mapping
from source.DQNAgent import DQNAgent
from IPython.display import clear_output
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
%matplotlib notebook

In [3]:
seasons = 1
Game = Regency(True)
clear_output()


In [4]:
Game.random_events()
Game.collect_regency_points()
Game.domain_initiative()
Game.collect_gold_bars()
Game.maintenance_costs(Update=True)

Instructions for updating:
Use tf.cast instead.


In [5]:
Game.add_holding('Gulfport', 'Ch', 'Source', 5)
Game.change_provence('Gulfport', Castle=3, Castle_Name='My Castle')
Game.add_troops(Regent='JR', Provence='Gulfport', Type='Archers')
Game.add_troops(Regent='JR', Provence='Gulfport', Type='Archers')
Game.add_troops(Regent='JR', Provence='Gulfport', Type='Archers')

In [6]:
Game.change_regent('Ch', Gold_Bars=200)
Game.change_regent('Ch', Regency_Points=200)

In [7]:
Game.set_override('Ch', 64, enemy='JR', provences=['Gulfport'])
Game.take_domain_actions()


Time to Finish the Rounditiative  0
0


In [8]:
df = Game.Seasons[0]['Actions'][1]
print(df[df['Regent'].str.contains('Ch')]['Message'].values[0])

Lacks the resources to cast Legion of the Dead


In [9]:
Game.Projects

,Regent,Project Type,Details,Gold Bars Left
